In [ ]:
# Load Pictures function
def DicLoadImages(path):
    import csv
    import re
    import os
    
    File = []
    with open(path) as csvfile:
        inputfile = csv.reader(csvfile)
        for row in inputfile:
            File.append(row)
            
    # Retrieve pictures name
    pics = []
    for pos in File:
        test = ','.join(pos)
        # For some reason I have to put all those dots to account for 8-character words, will need to fix this later.
        pathname = os.path.splitext(path)[0] + '/(.|..|...|....|.....|......|.......|........).png'
        pic = re.search(pathname,test)
        if pic:
            found = pic.group(1)
            pics.append(found)
            
    return File, pics

In [ ]:
def DicActivArray(i,picmatrix,pics):  
    import re
#     from radicals import dictionary_name
#     from radicals2 import dictionary_name #radicals2 is the one I made with only 5 motors
    from radicals3 import dictionary_name #radicals3 has 10 motors where the vowels have individual motors assigned
    # Initialize array to store testmatrix values
    new_array = []   
    # Retrieve word that is being used and extract name
#     test = ','.join(picmatrix[i])
#     pic = re.search('syllables/(.+?).png',test)
#     if pic:
#         found = pic.group(1)
#         current_array = list(found)
    current_array = pics[i]
    # Loop to extract letters
    for character in current_array:
        try:
            for item in dictionary_name[character]:
#                 new_array.append(item+4)
                new_array.append(item+2) # +2 because I added a list of items that goes from 1 to 10 
                new_array.append(70)
        except ValueError:
            pass
    
#     # With name retrieve dictionary key & add delays
#     current_array = dictionary_name[pics[i]]

#     for item in current_array:
#         try:
#             new_array.append(int(item)+4)
#             new_array.append(50)
#         except ValueError:
#             pass
#     print(new_array)
    return new_array

In [ ]:
# This is the function to emit to Serial but without display the pictures

def DicTestPattern(ser,i,picmatrix,pics):
    import struct
    import time
    
    # Read the incoming line
#     print(ser.readline())
    
    while(True):
    
    # If the incoming line matches the trigger
        if (ser.readline() == b'ready\r\n'):
#     if ser == 'hi':
        
        # Then send flag to Arduino to start listening
            ser.write(b'1') # Need to include the b so that data can be sent
                        # this is python 3 syntax
    
        # Get the current array for the pattern
            activ_array = DicActivArray(i,picmatrix,pics)
        # Read length of the array
            Counter = len(activ_array)
    
        # Send length of the array to Arduino so it knows how many to wait for
            ser.write(struct.pack('>B', Counter)) # Convert the integers to bits to send via Serial
#             print(struct.pack('>B', Counter)) Don't necessarily need to print this every time

        # Initialize dummy variable to loop through array
            c = 0
        # Loop through length of array sending values
            for y in range(Counter//2):
                ser.write(struct.pack('>B', activ_array[c]))
                ser.write(struct.pack('>B', activ_array[c+1]+10))
#                 print('Im sending %d %d' % (activ_array[c], activ_array[c+1])) Don't necessarily need to print this every time
            # Jump two to go to the next pair
                c = c+2
        # Define waiting length
#         time.sleep(1)
            break
        else:
            time.sleep(0.1)
    return 1

In [ ]:
def DicDisplayAndSendCommands(x,Chinesevoice,ser,pics,picmatrix,n,test_array):
    %run _Functions.ipynb

    import tkinter as tk
    from PIL import Image, ImageTk
    from pydub import AudioSegment
    from pydub.playback import play as pplay
    
    song = AudioSegment.from_file(','.join(voice[x]),format='m4a')
    def play():
        pplay(song)
        
    def senso():
        DicTestPattern(ser,x,picmatrix,pics)
        
    def ShowChoice():
        master.destroy()
        selection = v.get()
        return selection

    master = tk.Tk()
    # This line plays the tune after 500ms
    master.after(700, play)
    # This line sends the sensogram after 100ms, I begin this first because it takes time to send
    master.after(500, senso)
    
    # This line of code specifies where the window opens up and how big it is
    master.geometry("1900x500+300+300")
    v = tk.IntVar()
    
    img = []
    photo = []
    
    for i in range(n):
        img.append(Image.open(','.join(picmatrix[test_array[i]])))
        img[i] = img[i].resize((250,250),Image.NEAREST)
        photo.append(ImageTk.PhotoImage(img[i]))

        but = tk.Radiobutton(master, 
              text="Image",image=photo[i],
              padx = 20, 
              variable=v,indicatoron = 0,command=ShowChoice, 
              value=i)
        
        if i < 5:
            but.grid(row=0,column=i)
        else:
            but.grid(row=1,column=i-5)
    
    master.mainloop()
    return v.get()

In [ ]:
# This is the Testing Function and Storing values into the Results Matrix

def DicTestFunction(x,voice,ser,pics,picmatrix,order,n,t):
    import time
    %run _Functions.ipynb
# So I'm allowing the user to select between 1-10, because of the way they are pasted, I know which
# picture corresponds to each number. Knowing the picture name, I can correlate to the picture that 
# should have appeared with the pattern by comparing the names I guess. 

    # Generate the random array, choose out of 20 and pick 4
    test_array = PickRandomIndexes(x,order,t)
    
    selection = DicDisplayAndSendCommands(x,voice,ser,pics,picmatrix,n,test_array)

#     time.sleep(1)

    TestforthePicture(test_array,x,selection)
    
    return test_array, selection

In [ ]:
def DicSoundnCommand(x,voice,ser,pics,picmatrix,count):
    %run _Functions.ipynb

    import tkinter as tk
    from PIL import Image, ImageTk
    from pydub import AudioSegment
    from pydub.playback import play as pplay
    import pyphen
    
    song = AudioSegment.from_file(','.join(voice[x]),format='m4a')
    def play():
        pplay(song)
        
    def senso():
        DicTestPattern(ser,x,picmatrix,pics)
        
    def ShowChoice():
        master.destroy()
        selection = v.get()
        return selection

    master = tk.Tk()
    # This line plays the tune after 500ms
    master.after(100, play)
    # This line sends the sensogram after 100ms, I begin this first because it takes time to send
    master.after(50, senso)
    
    # This line of code specifies where the window opens up and how big it is
    master.geometry("1900x500+300+300")
    v = tk.IntVar()
    
    img = []
    photo = []
    
    if count == 2:
        # This line of code specifies where the window opens up and how big it is
        master.geometry("1900x500+300+300")
        v = tk.IntVar()
    
        img = []
        photo = []
    
        img.append(Image.open('YES.png'))
        img[0] = img[0].resize((250,250),Image.NEAREST)
        photo.append(ImageTk.PhotoImage(img[0]))
        img.append(Image.open('NO.png'))
        img[1] = img[1].resize((250,250),Image.NEAREST)
        photo.append(ImageTk.PhotoImage(img[1]))

        tk.Radiobutton(master, 
              text="YES",image=photo[0],
              padx = 20, 
              variable=v,indicatoron = 0,command = ShowChoice,
              value=1).pack(side=tk.LEFT)
        tk.Radiobutton(master, 
              text="NO",image=photo[1],
              padx = 20, 
              variable=v,indicatoron = 0,command = ShowChoice,
              value=2).pack(side=tk.LEFT)
    else:
        master.after(300,master.destroy)
        v = tk.IntVar()
        
    master.mainloop()
    return v.get()

In [20]:
def DicSound(x,voice,count):
    %run _Functions.ipynb

    import tkinter as tk
    from PIL import Image, ImageTk
    from pydub import AudioSegment
    from pydub.playback import play as pplay
    
    song = AudioSegment.from_file(','.join(voice[x]),format='m4a')
    def play():
        pplay(song)
        
    def ShowChoice():
        master.destroy()
        selection = v.get()
        return selection

    master = tk.Tk()
    # This line plays the tune after 500ms
#     master.after(700, play)
    master.after(100,play)
    if count == 2:
        # This line of code specifies where the window opens up and how big it is
        master.geometry("1900x500+300+300")
        v = tk.IntVar()
    
        img = []
        photo = []
    
        img.append(Image.open('YES.png'))
        img[0] = img[0].resize((250,250),Image.NEAREST)
        photo.append(ImageTk.PhotoImage(img[0]))
        img.append(Image.open('NO.png'))
        img[1] = img[1].resize((250,250),Image.NEAREST)
        photo.append(ImageTk.PhotoImage(img[1]))

        tk.Radiobutton(master, 
              text="YES",image=photo[0],
              padx = 20, 
              variable=v,indicatoron = 0,command = ShowChoice,
              value=1).pack(side=tk.LEFT)
        tk.Radiobutton(master, 
              text="NO",image=photo[1],
              padx = 20, 
              variable=v,indicatoron = 0,command = ShowChoice,
              value=2).pack(side=tk.LEFT)
    else:
        master.after(300,master.destroy)
        v = tk.IntVar()
        
    master.mainloop()
    return v.get()

In [4]:
def JaoderNein(string,selection,count):
    
    bigoku = [7,23,34]
    bulado = [9,35,13]
    tadupa = [65,14,50]
    datiba = [10,67,5]
    
    if (string == bigoku or string == bulado or string == tadupa or string == datiba) and selection == 1:           
        results.append(1)
    elif (string != bigoku or string != bulado or string != tadupa or string != datiba) and selection == 2:
        results.append(1)
    else:
        results.append(0)
        
    return results

In [1]:
# This is the Testing Function and Storing values into the Results Matrix for the 
# Saffran Experiment

#def DicTestSensoSaffran(x,voice,ser,pics,picmatrix,count,string):
def DicTestSensoSaffran(x,voice,ser,pics,picmatrix,count,string):
    import time
#     %run _Functions.ipynb
# So I'm allowing the user to select between 1-10, because of the way they are pasted, I know which
# picture corresponds to each number. Knowing the picture name, I can correlate to the picture that 
# should have appeared with the pattern by comparing the names I guess. 

# NEED TO MAKE A PICMATRIX CONTAINING THE
    
    selection = DicSoundnCommand(x,voice,ser,pics,picmatrix,count)

#     time.sleep(1)
    if count == 2:
        JaoderNein(string,selection,count)
        return selection
    else: 
        return

In [2]:
# This is the Testing Function and Storing values into the Results Matrix for the 
# Saffran Experiment

def DicTestSaffran(x,voice,pics,picmatrix,order,count,string):
    import time
#     %run _Functions.ipynb
# So I'm allowing the user to select between 1-10, because of the way they are pasted, I know which
# picture corresponds to each number. Knowing the picture name, I can correlate to the picture that 
# should have appeared with the pattern by comparing the names I guess. 
    
    selection = DicSound(x,voice,count)

#     time.sleep(1)
    if count == 2:
        JaoderNein(string,selection,count)
        return selection
    else: 
        return

In [5]:
import pyphen
dic = pyphen.Pyphen(lang='es')
a=dic.inserted('datipa')
a.split('-')

['da', 'ti', 'pa']